<a href="https://colab.research.google.com/github/hannahhb/AugLy/blob/main/vqe_protein_folding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a code implementation for a CVaR - VQE algorithm to give a solution to the protein folding problem.

# Initialisation

In [9]:
!pip install pennylane pennylane-qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pennylane as qml


In [1]:
# Defining hyperParams as per the given details
hyperParams = {
    'protein': 'APRLRFY',
    'turn2qubit': '0100q1qqqqqq',
    'numQubitsConfig': None,
    'numQubitsInteraction': 2,
    'interactionEnergy': None, # This needs to be defined based on the specific problem
}

# Calculating the number of configuration qubits
hyperParams['numQubitsConfig'] = sum(c == 'q' for c in hyperParams['turn2qubit'])


# Build MJ Interaction Matrix

In [6]:

def build_MJ_interactions(protein):
    """
    Create the Miyazawa-Jernigan (MJ) interaction energy matrix for the protein,
    specified with 1-letter amino acid codes.

    Args:
    protein (str): The protein sequence using 1-letter amino acid codes.

    Returns:
    numpy.ndarray: The interaction energy matrix for the given protein.
    """
    N = len(protein)
    mat = np.zeros((N, N))
    np.random.seed(29507)
    MJ = np.random.rand(20, 20) * -6
    MJ = np.triu(MJ) + np.triu(MJ, 1).T
    acids = ["C", "M", "F", "I", "L", "V", "W", "Y", "A", "G", "T", "S", "N", "Q", "D", "E", "H", "R", "K", "P"]
    acid2idx = {acid: idx for idx, acid in enumerate(acids)}

    for i in range(N):
        for j in range(N):
            mat[i, j] = MJ[acid2idx[protein[i]], acid2idx[protein[j]]]

    return mat


In [7]:
hyperParams['numQubitsConfig'] = sum(c == 'q' for c in hyperParams['turn2qubit'])
hyperParams['interactionEnergy'] = build_MJ_interactions(hyperParams['protein'])

# Function to calculate energy of folds

currently this code only implements a 1 nearest neighbor model for the amino acid chain

In [8]:

def exact_hamiltonian(bitstrings, hyperParams):
    """
    Compute the Hamiltonian for each bit string (i.e., the energy for each fold).
    This does not consider the Hch constraint from side-chains and the interaction term
    is only 1-nearest-neighbor (1-NN).
    """
    lambdaDis = 720    # Penalty for interaction distance
    lambdaLoc = 20     # Penalty for interaction location
    lambdaBack = 50    # Penalty for unphysical geometry

    energies = np.zeros(len(bitstrings))
    numBeads = len(hyperParams['protein'])

    for idx, bitstring in enumerate(bitstrings):
        config = hyperParams['turn2qubit']
        config = ''.join(bitstring[i] if c == 'q' else c for i, c in enumerate(config))
        turns = np.array([int(config[i:i+2], 2) for i in range(0, len(config), 2)])

        # Geometric Hamiltonian Hgc
        energies[idx] += lambdaBack * np.sum(turns[:-1] == turns[1:])

        # 1-NN Interaction Hamiltonian Hin
        currInteractionQubit = hyperParams['numQubitsConfig']
        for i in range(numBeads - 4):
            for j in range(i + 5, numBeads, 2):
                currInteractionQubit += 1
                if bitstring[currInteractionQubit] == '0':
                    continue

                interactionEnergy = hyperParams['interactionEnergy'][i, j]

                # Add the interaction energy
                energies[idx] += interactionEnergy

                # Compute distances between interacting beads
                deltaN_ij, deltaN_ir, deltaN_mj = np.zeros(4), np.zeros(4), np.zeros(4)
                for k in range(4):
                    deltaN_ij[k] = np.sum((-1)**np.arange(i, j) * (turns[i:j] == k))
                    deltaN_ir[k] = np.sum((-1)**np.arange(i, j-1) * (turns[i:j-1] == k))
                    deltaN_mj[k] = np.sum((-1)**np.arange(i+1, j) * (turns[i+1:j] == k))

                d_ij = np.linalg.norm(deltaN_ij)**2
                d_ir = np.linalg.norm(deltaN_ir)**2
                d_mj = np.linalg.norm(deltaN_mj)**2

                # Add penalty for distance not equal to 1
                energies[idx] += lambdaDis * (d_ij - 1)

                # Add penalty for unphysical nearest neighbour collisions
                energies[idx] += lambdaLoc * (2 - d_ir) + lambdaLoc * (2 - d_mj)

                if i-1 >= 0:
                    deltaN_mj = np.array([np.sum((-1)**np.arange(i-1, j) * (turns[i-1:j] == k)) for k in range(4)])
                    d_mj = np.linalg.norm(deltaN_mj)**2
                    energies[idx] += lambdaLoc * (2 - d_mj)

                if j+1 <= numBeads:
                    deltaN_ir = np.array([np.sum((-1)**np.arange(i, j+1) * (turns[i:j+1] == k)) for k in range(4)])
                    d_ir = np.linalg.norm(deltaN_ir)**2
                    energies[idx] += lambdaLoc * (2 - d_ir)

    return energies


Compute Minimum Energy for All Folds [TBD]


# Write CVaR-VQE Objective Function [TBD]

# Create Circuit Anstaz

In [12]:

def ProteinConfigAnsatz(parameters):
    """
    Create the circuit ansatz for a 7 amino acid neuropeptide (10 qubit circuit) with adjusted parameters handling.
    """
    parameters = np.reshape(parameters, (2, 9))

    # Ensure we have the correct device setup
    n_qubits = 10
    dev = qml.device('default.qubit', wires=n_qubits)

    @qml.qnode(dev)
    def circuit(params):
        # Apply Hadamard gates to all qubits
        for i in range(n_qubits):
            qml.Hadamard(wires=i)

        # Apply the first layer of RY gates based on the parameters
        # Adjusting for the missing 10th parameter by using the last parameter for the 9th and 10th qubit
        for i in range(n_qubits - 1):  # Apply parameters to first 9 qubits directly
            qml.RY(params[0, i], wires=i)
        qml.RY(params[0, -1], wires=9)  # Use the last parameter for the 10th qubit as well

        # Define CNOT gates as specified
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4])
        qml.CNOT(wires=[4, 9])
        qml.CNOT(wires=[9, 8])
        qml.CNOT(wires=[8, 7])
        # The following CNOT gates repeat some operations, ensure this is intended
        qml.CNOT(wires=[7, 8])
        qml.CNOT(wires=[8, 7])
        qml.CNOT(wires=[7, 6])
        qml.CNOT(wires=[6, 5])
        qml.CNOT(wires=[5, 0])

        # Apply the second layer of RY gates with parameters, similarly handling the 10th qubit
        for i in range(n_qubits - 1):  # Apply parameters to first 9 qubits directly
            qml.RY(params[1, i], wires=i)
        qml.RY(params[1, -1], wires=9)  # Use the last parameter for the 10th qubit as well

        return qml.state()

    return circuit

# Generate random parameters for the ansatz, with an adjustment for the 10th qubit
params = np.random.rand(2, 9)

# Create the ansatz circuit
ansatz = ProteinConfigAnsatz(params)

# Drawing the circuit to visualize its structure
print(qml.draw(ansatz)(params))


0: ──H──RY(0.74)─╭●────────────────────────────────────────────────────╭X─────────RY(0.34)─┤  State
1: ──H──RY(0.91)─╰X─╭●─────────────────────────────────────────────────│──────────RY(0.95)─┤  State
2: ──H──RY(0.46)────╰X─╭●──────────────────────────────────────────────│──────────RY(0.96)─┤  State
3: ──H──RY(0.26)───────╰X─╭●───────────────────────────────────────────│──────────RY(0.38)─┤  State
4: ──H──RY(0.56)──────────╰X─╭●────────────────────────────────────────│──────────RY(0.10)─┤  State
5: ──H──RY(0.09)─────────────│───────────────────────────────╭X────────╰●─────────RY(0.75)─┤  State
6: ──H──RY(0.52)─────────────│─────────────────────╭X────────╰●─────────RY(0.34)───────────┤  State
7: ──H──RY(0.19)─────────────│─────╭X────────╭●─╭X─╰●─────────RY(0.15)─────────────────────┤  State
8: ──H──RY(0.67)─────────────│──╭X─╰●────────╰X─╰●──RY(0.32)───────────────────────────────┤  State
9: ──H──RY(0.67)─────────────╰X─╰●──RY(0.32)───────────────────────────────────────────────┤  State


# Acknowledgements

Code is adapted from [MATLAB Community tutorial](https://www.mathworks.com/help/matlab/math/ground-state-protein-folding-using-variational-quantum-eigensolver-vqe.html#mw_d653153e-02d5-4350-8476-448e19c5b005) based on the paper [Resource-Efficient Quantum Algorithm for Protein Folding](https://arxiv.org/abs/1908.02163) Robert et al. from IBM Zurich Team.